# Task3 Deep Learning for Streamline Segmentation from Earth imagery
<font color="red">Instructions: please fill in marked missing codes and answer required questions. </font> 
## Part 1: Loading Data
There are six data files in the folder:
>X_test.npy   X_val.npy   Y_train.npy  
X_train.npy  Y_test.npy  Y_val.npy

They are corresponding to the feature images X and label images Y in training set, validation set, and test set.

In the feature images, there are 7 channels, including:

**Red, Green, Blue, NIR (near infrared), DEM (digital elementation), lidar intensity, Slope.**

If you want to use Google Colab and request its computing resources, please download the provided dataset and upload it to your Google Drive folder. Then mount the Google Drive folder to your Colab virtual machine. Please refer to the following link for details: https://colab.research.google.com/notebooks/io.ipynb

In [ ]:
# Import packages
import os, sys, time,tempfile, random, csv
import tensorflow as tf
import keras 
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import concatenate

import keras.backend as K
import keras.metrics as kmetrics
from keras.layers import UpSampling2D
from keras.layers import BatchNormalization
from keras.layers.core import SpatialDropout2D, Activation
from keras.layers.convolutional import Conv2D, Conv2DTranspose
from keras.layers.pooling import MaxPooling2D
from keras.layers import Input
from keras.layers.core import Dropout, Lambda
from keras.models import Model, save_model, load_model
import matplotlib.pyplot as plt
from keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau, TensorBoard
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

In [ ]:
### Loading Tensors
X_train = np.load('X_train.npy')
Y_train = np.load('Y_train.npy')

X_val = np.load('X_val.npy')
Y_val = np.load('Y_val.npy')

X_test = np.load('X_test.npy')
Y_test = np.load('Y_test.npy')

print("Successfully loaded tensors.")

PLEASE ANSWER THE QUESTIONS BELOW: 
1. How many training, validation, and test images, respectively?
2. What is the size (width, height) of each image?
3. How many channels are there in each feature image?

You can answer above questions with variables defined above.

In [ ]:
# print out your answers below

print('The number of training, validation, and test images is', )

print('The size of each image is: ', )

print('The channels number in each image is: ', )

In [ ]:
# randomly sample a number of training images
# plot their RGB features and DEM (digital elemention model) feature, and ground truth pixel labels
# note that some image patches may not have class 1 (stream) pixels (these are image windows without a stream passing through)
sample_count = 3

for i in range(sample_count):
    t_choice = np.random.randint(0, len(X_train)-1)
    
    y_tile = np.squeeze(Y_train[t_choice])
    
    x_tile = X_train[t_choice]
    
    fig, (ax1, ax2, ax3) = plt.subplots(ncols=3, figsize=(9,4))
    
    print('Sample {}'.format(t_choice))
    ax1.imshow(x_tile[:,:,0:3])
    ax1.set_title('Train Image {}'.format(t_choice))
        
    ax2.imshow(x_tile[:,:,4])
    ax2.set_title('Train DEM {}'.format(t_choice))
    
    ax3.imshow(y_tile)
    ax3.set_title('Train Label {}'.format(t_choice))
    
    plt.show()

## Part 2: Model Construction

This part defines the U-Net model architecture, defines the **negative Dice Coefficient** as the loss function, and defines several evaluation metric functions, e.g., precision, recall, F1-score.

Below is the U-Net model architecture used in Lab 3. 

In [ ]:
###U-Net model architecture
############################

INPUT_CHANNELS = 7
OUTPUT_CHANNELS = 1 

def double_conv_layer(x, size, std_init, dropout=0.2):
    if K.image_data_format() == 'channels_first':
        axis = 1
    else:
        axis = 3
        
    # TODO: Implement double convolutional layer (Conv2D) below with 3*3 convolutional kernel, 
    # padding to make the output size be the same as the input size, and select ReLU as the activation function (Activation).
    # Note: Consecutive 2 convolutional layers with no pooling.
    conv_1 = 
    conv_1_activated = 
    conv_2 = 
    conv_2_activated = 
    # TODO Ends
    
    if dropout > 0:
        conv = SpatialDropout2D(dropout)(conv_2_activated)
    return conv

def UNET_7_224_dropout(dropout_val=0.2, std_init=None):
    if K.image_data_format() == 'channels_first':
        inputs = Input((INPUT_CHANNELS, 224, 224))
        axis = 1
    else:
        inputs = Input((224, 224, INPUT_CHANNELS))
        axis = 3
    filters = 32

    conv_224 = double_conv_layer(inputs, filters, std_init)
    pool_112 = MaxPooling2D(pool_size=(2, 2))(conv_224)

    conv_112 = double_conv_layer(pool_112, 2*filters, std_init)
    pool_56 = MaxPooling2D(pool_size=(2, 2))(conv_112)

    conv_56 = double_conv_layer(pool_56, 4*filters, std_init)
    pool_28 = MaxPooling2D(pool_size=(2, 2))(conv_56)

    conv_28 = double_conv_layer(pool_28, 8*filters, std_init)
    pool_14 = MaxPooling2D(pool_size=(2, 2))(conv_28)

    conv_14 = double_conv_layer(pool_14, 16*filters, std_init)
    pool_7 = MaxPooling2D(pool_size=(2, 2))(conv_14)

    conv_7 = double_conv_layer(pool_7, 32*filters, std_init)

    up_14 = concatenate([UpSampling2D(size=(2, 2))(conv_7), conv_14], axis=axis)
    up_conv_14 = double_conv_layer(up_14, 16*filters, std_init,dropout_val)

    up_28 = concatenate([UpSampling2D(size=(2, 2))(up_conv_14), conv_28], axis=axis)
    up_conv_28 = double_conv_layer(up_28, 8*filters, std_init,dropout_val)

    up_56 = concatenate([UpSampling2D(size=(2, 2))(up_conv_28), conv_56], axis=axis)
    up_conv_56 = double_conv_layer(up_56, 4*filters, std_init,dropout_val)

    up_112 = concatenate([UpSampling2D(size=(2, 2))(up_conv_56), conv_112], axis=axis)
    up_conv_112 = double_conv_layer(up_112, 2*filters, std_init,dropout_val)

    up_224 = concatenate([UpSampling2D(size=(2, 2))(up_conv_112), conv_224], axis=axis)
    up_conv_224 = double_conv_layer(up_224, filters, std_init, dropout_val)

    conv_final = Conv2D(OUTPUT_CHANNELS, (1, 1), kernel_initializer=std_init)(up_conv_224)
    
    # TODO: Choose an appropriate activation function.
    conv_final = Activation(' ')(conv_final)
    # TODO Ends

    model = Model(inputs, conv_final, name="UNET_7_224")

    return model

PLEASE ANSWER THE QUESTIONS BELOW: 
1. How many downsample (MaxPooling2D) operations and upsampling (UpSampling2D) operations in total, respectively?
2. How many convolution (Conv2D) operations in total?
3. What is the non-linear activation function used for the final class layer, sigmoid or softmax? Why is this function chosen (but not the other)?


In [ ]:
# My answer is:
ans = '''
1. xxx \n
2. xxx \n
3. xxx
'''

print(ans)

**Below is the loss function, i.e., negative dice coefficience.**


In [ ]:
### Loss function, negative dice coefficient
#################

def dice_coef(y_true, y_pred):
    """ Soft label dice coefficient measure. """
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    intersection = K.sum(y_true_f * y_pred_f)
    return (2.0 * intersection + 1.0) / (K.sum(y_true_f) + K.sum(y_pred_f) + 1.0)

def dice_coef_loss(y_true, y_pred):
    return - dice_coef(y_true, y_pred)

### Part 3: Model Training and Evaluation


In [ ]:
### Model Training
num_epochs = 30
batch_size = 32
learn_rate = 0.01

model = UNET_7_224_dropout(dropout_val=0.2)
model.compile(optimizer=Adam(learning_rate=learn_rate, epsilon=1e-8, weight_decay=1e-5), loss=dice_coef_loss, metrics=['accuracy'])

my_callbacks = [ 
      ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=5, min_lr=1e-5, min_deta=0.00001, verbose=0, mode='min'),
      EarlyStopping(monitor='val_loss', patience=20, verbose=0)]

# TODO: Complete the fit function with parameters (num_epochs, batch_size, my_callbacks) defined above.
# Note: The vairable "results" will store information about the training process.
# Place your code here (1 line)
results = 
# TODO Ends

In [ ]:
model.summary()

Plot training and validation curves

**Hint**: the training process has quite some randomness. If your curve shape does not look good, you can re-run the training codes and check again.

In [ ]:
plt.figure()
plt.plot(results.history['loss'],label='training loss')
plt.plot(results.history['val_loss'],label='validation loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend(loc='upper right')

PLEASE ANSWER THE QUESTIONS BELOW: Based on the figure above, has the loss function converged? Do you think it's necessary to continue training? Please explain your reasoning.

In [ ]:
ans = '''
xxx
'''

print(ans)

### Evaluation metrics

Below is the evaluation metrics (precision, recall, and f1-score) on Y predictions.

In [ ]:
### Model Metrics
#################
""" y_true is binary with shape [xx,224,224,1] and y_pred is probabilities with the same shape """

def f1_score(y_true, y_pred): 
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    recall = true_positives / (possible_positives + K.epsilon())
    f1_val = 2 * (precision * recall) / (precision + recall + K.epsilon())
    return f1_val

def precision(y_true, y_pred):
    """ Get precision """
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

def recall(y_true, y_pred):
    """ Get recall """
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

In [ ]:
""" Evaluate Model Preformance """
print("Preformance:")

# TODO: implement the codes for evaluation below.
# Training Performance
train_precision = 
train_recall = 
train_f1 = 
print("Training precision: {:.2f}\n".format( train_precision ) , 
      "Training Recall: {:.2f}\n".format(train_recall) , 
      "Training F1 Score: {:.2f}\n".format(train_f1) )

# Validation Performance
val_precision = 
val_recall = 
val_f1 = 
print("Validation precision: {:.2f}\n ".format(val_precision) , 
      "Validation Recall: {:.2f}\n".format(val_recall) , 
      "Validation F1 Score: {:.2f}\n".format(val_f1) )

# Test Performance
test_precision = 
test_recall =  
test_f1 = 
print("Test precision: {:.2f}\n ".format(test_precision) , 
      "Test Recall: {:.2f}\n".format(test_recall), 
      "Test F1 Score: {:.2f}\n".format(test_f1))

# TODO Ends


### Result Interpretation

Visualize the prediction results and compare them with feature images and ground truth class labels.

In [ ]:
Y_pred = np.copy(model.predict(X_test))
sample_count = 3
for i in range(sample_count):
    t_choice = np.random.randint(0, len(X_test)-1)
    
    y_tile = np.squeeze(Y_test[t_choice])

    y_pred_i = np.squeeze(Y_pred[t_choice])

    y_pred_i = np.where(y_pred_i>=0.5, 1, 0)
    
    x_tile = X_test[t_choice]
    
    fig, (ax1, ax2, ax3, ax4) = plt.subplots(ncols=4, figsize=(9,4))
    
    print('\n\n\nSample {}'.format(t_choice))
    ax1.imshow(x_tile[:,:,0:3])
    ax1.set_title('Test Image {}'.format(t_choice))
        
    ax2.imshow(x_tile[:,:,4])
    ax2.set_title('Test DEM {}'.format(t_choice))
    
    ax3.imshow(y_tile)
    ax3.set_title('Test Truth Label {}'.format(t_choice))
    
    ax4.imshow(y_pred_i)
    ax4.set_title('Test Predicted Label {}'.format(t_choice))

    plt.show()

PLEASE ANSWER THE QUESTIONS BELOW: What is your analysis of the results from both the quantitative metrics (precision, recall, f1-score) and qualitative visualization? </font>

In [ ]:
ans = '''
xxx
'''

print(ans)